In [1]:
from random import randrange
from csv import reader
from math import sqrt

In [2]:
# Converting data into CSV format
import pandas as pd

f = open("yeast.data")
data = f.read()
data = data.split("\n")
mydict = {"index":[],"1":[],"2":[],"3":[],"4":[],"5":[],"6":[],"7":[],"8":[],"9":[]}
df = pd.DataFrame(mydict)
for k in data[:-1]:
    row = []
    temp = k.split()
    for i in range(0,len(temp)):  
        row.append(temp[i])
    df.loc[len(df.index)] = row

df = df.drop("index", axis=1)
df.to_csv("yeast.csv", index=False, header=False)

In [3]:
# Load a CSV file
def load_csv(filename):
    dataset = list()
    with open(filename, 'r') as file:
        csv_reader = reader(file)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
    return dataset

# String -> Float
def str_column_to_float(dataset, column):
    for row in dataset:
        row[column] = float(row[column].strip())

# String -> integer
def str_column_to_int(dataset, column):
    class_values = [row[column] for row in dataset]
    unique = set(class_values)
    lookup = dict()
    for i, value in enumerate(unique):
        lookup[value] = i
    for row in dataset:
        row[column] = lookup[row[column]]
    return lookup

# min and max search
def dataset_minmax(dataset):
    minmax = list()
    for i in range(len(dataset[0])):
        col_values = [row[i] for row in dataset]
        value_min = min(col_values)
        value_max = max(col_values)
        minmax.append([value_min, value_max])
    return minmax

# Normalize the dataset
def normalize_dataset(dataset, minmax):
    for row in dataset:
        for i in range(len(row)):
            row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])
            
# Cross validation k folds
def cross_validation_split(dataset, n_folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for _ in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split

# Accuracy calculating
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

# Combining functions in evaluate
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    scores = list()
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        test_set = list()
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        accuracy = accuracy_metric(actual, predicted)
        scores.append(accuracy)
    return scores

# Eucliden distance calculator
def euclidean_distance(row1, row2):
    distance = 0.0
    for i in range(len(row1)-1):
        distance += (row1[i] - row2[i])**2
    return sqrt(distance)

# Find closest points
def get_neighbors(train, test_row, num_neighbors):
    distances = list()
    for train_row in train:
        dist = euclidean_distance(test_row, train_row)
        distances.append((train_row, dist))
    distances.sort(key=lambda tup: tup[1])
    neighbors = list()
    for i in range(num_neighbors):
        neighbors.append(distances[i][0])
    return neighbors

# Prediction
def predict_classification(train, test_row, num_neighbors):
    neighbors = get_neighbors(train, test_row, num_neighbors)
    output_values = [row[-1] for row in neighbors]
    prediction = max(set(output_values), key=output_values.count)
    return prediction

# KNN with Euclidean distance
def k_nearest_neighbors(train, test, num_neighbors):
    predictions = list()
    for row in test:
        output = predict_classification(train, row, num_neighbors)
        predictions.append(output)
    return(predictions)

In [4]:
# Test the kNN on the Yeast dataset with Euclidean distance
filename = 'yeast.csv'
dataset = load_csv(filename)
for i in range(len(dataset[0])-1):
    str_column_to_float(dataset, i)
# convert class column to integers
str_column_to_int(dataset, len(dataset[0])-1)
# evaluate algorithm
n_folds = 5
for i in range(5,21):
    num_neighbors = i
    scores = evaluate_algorithm(dataset, k_nearest_neighbors, n_folds, num_neighbors)
    print()
    print('Scores: %s' % scores)
    print("Mean accuracy at number of neighbours = ",i)
    print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))


Scores: [54.054054054054056, 57.432432432432435, 52.36486486486487, 58.108108108108105, 60.47297297297297]
Mean accuracy at number of neighbours =  5
Mean Accuracy: 56.486%

Scores: [58.108108108108105, 53.71621621621622, 53.71621621621622, 60.13513513513513, 57.432432432432435]
Mean accuracy at number of neighbours =  6
Mean Accuracy: 56.622%

Scores: [55.4054054054054, 55.4054054054054, 57.0945945945946, 59.797297297297305, 52.36486486486487]
Mean accuracy at number of neighbours =  7
Mean Accuracy: 56.014%

Scores: [57.0945945945946, 54.391891891891895, 57.0945945945946, 61.82432432432432, 59.797297297297305]
Mean accuracy at number of neighbours =  8
Mean Accuracy: 58.041%

Scores: [61.14864864864865, 52.702702702702695, 54.391891891891895, 55.067567567567565, 56.75675675675676]
Mean accuracy at number of neighbours =  9
Mean Accuracy: 56.014%

Scores: [54.729729729729726, 57.770270270270274, 59.797297297297305, 58.78378378378378, 60.13513513513513]
Mean accuracy at number of neig

In [5]:
# Load a CSV file
def load_csv(filename):
    dataset = list()
    with open(filename, 'r') as file:
        csv_reader = reader(file)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
    return dataset

# String -> Float
def str_column_to_float(dataset, column):
    for row in dataset:
        row[column] = float(row[column].strip())

# String -> integer
def str_column_to_int(dataset, column):
    class_values = [row[column] for row in dataset]
    unique = set(class_values)
    lookup = dict()
    for i, value in enumerate(unique):
        lookup[value] = i
    for row in dataset:
        row[column] = lookup[row[column]]
    return lookup

# min and max search
def dataset_minmax(dataset):
    minmax = list()
    for i in range(len(dataset[0])):
        col_values = [row[i] for row in dataset]
        value_min = min(col_values)
        value_max = max(col_values)
        minmax.append([value_min, value_max])
    return minmax

# Normalize the dataset
def normalize_dataset(dataset, minmax):
    for row in dataset:
        for i in range(len(row)):
            row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])
            
# Cross validation k folds
def cross_validation_split(dataset, n_folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for _ in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split

# Accuracy calculating
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

# Combining functions in evaluate
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    scores = list()
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        test_set = list()
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        accuracy = accuracy_metric(actual, predicted)
        scores.append(accuracy)
    return scores

# Eucliden distance calculator
def manhattan_distance(row1, row2):
    distance = 0.0
    for i in range(len(row1)-1):
        distance += abs(row1[i] - row2[i])
    return sqrt(distance)

# Find closest points
def get_neighbors(train, test_row, num_neighbors):
    distances = list()
    for train_row in train:
        dist = manhattan_distance(test_row, train_row)
        distances.append((train_row, dist))
    distances.sort(key=lambda tup: tup[1])
    neighbors = list()
    for i in range(num_neighbors):
        neighbors.append(distances[i][0])
    return neighbors

# Prediction
def predict_classification(train, test_row, num_neighbors):
    neighbors = get_neighbors(train, test_row, num_neighbors)
    output_values = [row[-1] for row in neighbors]
    prediction = max(set(output_values), key=output_values.count)
    return prediction

# KNN with Manhattan distance
def k_nearest_neighbors(train, test, num_neighbors):
    predictions = list()
    for row in test:
        output = predict_classification(train, row, num_neighbors)
        predictions.append(output)
    return(predictions)

In [6]:
# Test the kNN on the Yeast dataset with Manhattan distance
filename = 'yeast.csv'
dataset = load_csv(filename)
for i in range(len(dataset[0])-1):
    str_column_to_float(dataset, i)
# convert class column to integers
str_column_to_int(dataset, len(dataset[0])-1)
# evaluate algorithm
n_folds = 5
for i in range(5,21):
    num_neighbors = i
    scores = evaluate_algorithm(dataset, k_nearest_neighbors, n_folds, num_neighbors)
    print()
    print('Scores: %s' % scores)
    print("Mean accuracy at number of neighbours = ",i)
    print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))


Scores: [62.83783783783784, 58.78378378378378, 58.445945945945944, 53.71621621621622, 55.067567567567565]
Mean accuracy at number of neighbours =  5
Mean Accuracy: 57.770%

Scores: [56.08108108108109, 61.14864864864865, 59.797297297297305, 56.41891891891891, 55.74324324324324]
Mean accuracy at number of neighbours =  6
Mean Accuracy: 57.838%

Scores: [59.797297297297305, 59.12162162162162, 55.74324324324324, 55.067567567567565, 55.4054054054054]
Mean accuracy at number of neighbours =  7
Mean Accuracy: 57.027%

Scores: [60.810810810810814, 59.45945945945946, 55.067567567567565, 55.74324324324324, 56.75675675675676]
Mean accuracy at number of neighbours =  8
Mean Accuracy: 57.568%

Scores: [62.83783783783784, 57.770270270270274, 58.445945945945944, 56.41891891891891, 56.41891891891891]
Mean accuracy at number of neighbours =  9
Mean Accuracy: 58.378%

Scores: [56.08108108108109, 61.82432432432432, 59.45945945945946, 54.391891891891895, 59.797297297297305]
Mean accuracy at number of nei